In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [4]:
# Load Train, Validation, Test Dataset
train_dataset = pd.read_csv('../saveDir/MP_IN_adm_train.csv')
val_dataset = pd.read_csv('../saveDir//MP_IN_adm_val.csv')
test_dataset = pd.read_csv('../saveDir/MP_IN_adm_test.csv')

In [5]:
from transformers import AutoModelForSequenceClassification, AutoConfig

# Specify the dropout rate in the configuration
config = AutoConfig.from_pretrained('bvanaken/CORe-clinical-outcome-biobert-v1', 
                                    num_labels=2, 
                                    hidden_dropout_prob=0.2, 
                                    attention_probs_dropout_prob=0.2)

# Load the pre-trained model with the specified configuration
core_model = AutoModelForSequenceClassification.from_pretrained('bvanaken/CORe-clinical-outcome-biobert-v1', config=config)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bvanaken/CORe-clinical-outcome-biobert-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# from transformers import AutoModelForSequenceClassification, AutoConfig

# # Specify the dropout rate in the configuration
# config = AutoConfig.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', 
#                                     num_labels=4, 
#                                     hidden_dropout_prob=0.2, 
#                                     attention_probs_dropout_prob=0.2)

# # Load the pre-trained model with the specified configuration
# clinical_model = AutoModelForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', config=config)

In [7]:
# from transformers import AutoModelForSequenceClassification, AutoConfig

# # Specify the dropout rate in the configuration
# config = AutoConfig.from_pretrained('dmis-lab/biobert-base-cased-v1.2', 
#                                     num_labels=4, 
#                                     hidden_dropout_prob=0.2, 
#                                     attention_probs_dropout_prob=0.2)

# # Load the pre-trained model with the specified configuration
# base_model = AutoModelForSequenceClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.2', config=config)

In [8]:
# from transformers import AutoModelForSequenceClassification, AutoConfig

# # Specify the dropout rate in the configuration
# config = AutoConfig.from_pretrained('emilyalsentzer/Bio_Discharge_Summary_BERT', 
#                                     num_labels=2, 
#                                     hidden_dropout_prob=0.2, 
#                                     attention_probs_dropout_prob=0.2)

# # Load the pre-trained model with the specified configuration
# discharge_model = AutoModelForSequenceClassification.from_pretrained('emilyalsentzer/Bio_Discharge_Summary_BERT', config=config)

In [9]:
from transformers import AutoTokenizer

# Choose a tokenizer
tokenizer = AutoTokenizer.from_pretrained('bvanaken/CORe-clinical-outcome-biobert-v1')

In [10]:
from transformers import BertTokenizer, BertModel
import torch

def split_inputs(input_ids, attention_mask, tokenizer):
    # Get the special token ids
    cls_token_id = tokenizer.cls_token_id
    sep_token_id = tokenizer.sep_token_id

    # Split the tensors
    input_ids1 = input_ids[:, :510]
    input_ids2 = input_ids[:, 460:970]

    attention_mask1 = attention_mask[:, :510]
    attention_mask2 = attention_mask[:, 460:970]

    # Add [CLS] and [SEP] tokens
    input_ids1 = torch.cat([torch.full_like(input_ids1[:, :1], cls_token_id), input_ids1, torch.full_like(input_ids1[:, :1], sep_token_id)], dim=1)
    input_ids2 = torch.cat([torch.full_like(input_ids2[:, :1], cls_token_id), input_ids2, torch.full_like(input_ids2[:, :1], sep_token_id)], dim=1)

    attention_mask1 = torch.cat([torch.full_like(attention_mask1[:, :1], 1), attention_mask1, torch.full_like(attention_mask1[:, :1], 1)], dim=1)
    attention_mask2 = torch.cat([torch.full_like(attention_mask2[:, :1], 1), attention_mask2, torch.full_like(attention_mask2[:, :1], 1)], dim=1)

    return (input_ids1, attention_mask1), (input_ids2, attention_mask2)

In [11]:
from torch.utils.data import DataLoader
from torch import nn

class EnsembleModel(nn.Module):
    def __init__(self, model1):
        super(EnsembleModel, self).__init__()
        self.model1 = model1

    def forward(self, input_ids, attention_mask):
        # Assuming your batch size (18) tensors are input_ids and attention_mask
        (input_ids1, attention_mask1), (input_ids2, attention_mask2) = split_inputs(input_ids, attention_mask, tokenizer)

        outputs1 = self.model1(input_ids1, attention_mask=attention_mask1)[0]
        outputs2 = self.model1(input_ids2, attention_mask=attention_mask2)[0]
        
        avg_output = (outputs1 + outputs2) / 2.00
        return avg_output

In [12]:
# Apply the tokenizer to the training, validation, and test datasets
train_encodings = tokenizer(train_dataset['text'].tolist(), truncation=True, padding=True, max_length = 970)
val_encodings = tokenizer(val_dataset['text'].tolist(), truncation=True, padding=True,  max_length = 970)
test_encodings = tokenizer(test_dataset['text'].tolist(), truncation=True, padding=True , max_length = 970)

In [13]:
# Create a Dataset for PyTorch
class LosDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [14]:
train_dataset = LosDataset(train_encodings, train_dataset['hospital_expire_flag'].tolist())
val_dataset = LosDataset(val_encodings, val_dataset['hospital_expire_flag'].tolist())
test_dataset = LosDataset(test_encodings, test_dataset['hospital_expire_flag'].tolist())

In [15]:
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from tqdm import tqdm
from torch import nn
import numpy as np

# Create the ensemble model
ensemble_model = EnsembleModel(core_model)

In [16]:
import os

# list all files in the current directory
files = os.listdir('.')

# filter the ones that start with 'CORE_baseline'
core_models = [f for f in files if f.startswith('CORE_chunk_MP_over')]

if core_models:
    print("Found models starting with 'CORE_chunk_MP_over':")
    for model in core_models:
        print(model)
        
    # get the first (and supposedly only) model
    model_path = core_models[0]

    # load the model state
    ensemble_model.load_state_dict(torch.load(model_path))
    print("Loaded Model")
else:
    print("No models found starting with 'CORE_chunk_MP_over'.")

No models found starting with 'CORE_chunk_MP_over'.


In [17]:
# ensemble_model

In [18]:
# Push the model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ensemble_model = ensemble_model.to(device)

In [19]:
train_loader = DataLoader(train_dataset, batch_size=9, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=9, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=9, shuffle=False)

In [20]:
epochs = 200
best_roc_auc = 0.0
min_delta = 0.0001
early_stopping_count = 0
early_stopping_patience = 3
gradient_accumulation_steps = 10
best_model_path = "best_model.pth"

# Set the optimizer
optimizer = AdamW(ensemble_model.parameters(), lr=1e-5, weight_decay=0.01)

# Set the scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=50, 
    num_training_steps=len(train_loader) * epochs // gradient_accumulation_steps
)


/home/ec2-user/anaconda3/lib/python3.7/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [21]:
from torch.nn import functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Training
for epoch in range(epochs):
    ensemble_model.train()
    train_loss = 0
    for step, batch in enumerate(tqdm(train_loader)):
        optimizer.zero_grad() if step % gradient_accumulation_steps == 0 else None
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = ensemble_model(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        (loss / gradient_accumulation_steps).backward()
        train_loss += loss.item()
        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_loader):
            optimizer.step()
            scheduler.step()

    ensemble_model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = ensemble_model(input_ids, attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            val_loss += loss.item()
            val_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
            val_labels.append(labels.cpu().numpy())
            
    val_preds = np.concatenate(val_preds)
    val_labels = np.concatenate(val_labels)
    val_loss /= len(val_loader)
    train_loss /= len(train_loader)
    print(f'Epoch: {epoch+1}/{epochs}, Training Loss: {train_loss}, Validation Loss: {val_loss}')

    # Calculate metrics
    val_preds_class = np.argmax(val_preds, axis=1)
    accuracy = accuracy_score(val_labels, val_preds_class)
    recall = recall_score(val_labels, val_preds_class)
    precision = precision_score(val_labels, val_preds_class)
    f1 = f1_score(val_labels, val_preds_class)
    roc_auc = roc_auc_score(val_labels, val_preds[:, 1])

    print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Roc Auc: {roc_auc}')

    # Implement early stopping
    if epoch > 0 and roc_auc - best_roc_auc < min_delta:
        early_stopping_count += 1
        print(f'EarlyStopping counter: {early_stopping_count} out of {early_stopping_patience}')
        if early_stopping_count >= early_stopping_patience:
            print('Early stopping')
            break
    else:
        best_roc_auc = roc_auc
        early_stopping_count = 0
        torch.save(ensemble_model.state_dict(), f"CORE_chunk_MP_over_epoch_{epoch}roc_{best_roc_auc}.pth")


100%|██████████| 546/546 [04:56<00:00,  1.84it/s]
/home/ec2-user/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 1/200, Training Loss: 0.35287377570123624, Validation Loss: 0.31265125682939104
Accuracy: 0.8944580277098615, Recall: 0.0, Precision: 0.0, F1: 0.0, Roc Auc: 0.7228115847705824


100%|██████████| 546/546 [04:53<00:00,  1.86it/s]


Epoch: 2/200, Training Loss: 0.3001460176841977, Validation Loss: 0.28796655550298894
Accuracy: 0.8950692746536267, Recall: 0.03474903474903475, Precision: 0.5454545454545454, F1: 0.06533575317604355, Roc Auc: 0.7965941372547295


100%|██████████| 546/546 [04:35<00:00,  1.98it/s]


...

Epoch: 6/200, Training Loss: 0.2269435872032788, Validation Loss: 0.28053580923176513
Accuracy: 0.9011817440912795, Recall: 0.19884169884169883, Precision: 0.5953757225433526, F1: 0.2981186685962373, Roc Auc: 0.8261325757908902
EarlyStopping counter: 1 out of 3


100%|██████████| 546/546 [04:41<00:00,  1.94it/s]


Epoch: 7/200, Training Loss: 0.21172765144506128, Validation Loss: 0.28426721576522807
Accuracy: 0.8989405052974735, Recall: 0.21042471042471042, Precision: 0.5561224489795918, F1: 0.3053221288515406, Roc Auc: 0.8262574647540479
EarlyStopping counter: 2 out of 3


100%|██████████| 546/546 [04:52<00:00,  1.87it/s]

Epoch: 8/200, Training Loss: 0.19397499051740852, Validation Loss: 0.30235193184348363
Accuracy: 0.8954767726161369, Recall: 0.2548262548262548, Precision: 0.5096525096525096, F1: 0.33976833976833976, Roc Auc: 0.8178278994907697
EarlyStopping counter: 3 out of 3
Early stopping


In [22]:
import os

# list all files in the current directory
files = os.listdir('.')

# filter the ones that start with 'CORE_baseline'
core_models = [f for f in files if f.startswith('CORE_chunk_MP_over')]

if core_models:
    print("Found models starting with 'CORE_chunk_MP_over':")
    for model in core_models:
        print(model)
        
    # get the first (and supposedly only) model
    model_path = core_models[0]

    # load the model state
    ensemble_model.load_state_dict(torch.load(model_path))
    print("Loaded Model")
else:
    print("No models found starting with 'CORE_chunk_MP_over'.")

Found models starting with 'CORE_chunk_MP_over':
CORE_chunk_MP_over_epoch_4roc_0.8293541833405158.pth
Loaded Model


In [23]:
from torch.nn import functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Put the model in evaluation mode
ensemble_model.eval()

# Initialize lists to store predictions and true labels
test_preds = []
test_labels = []

# Iterate over test data
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = ensemble_model(input_ids, attention_mask)
        test_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
        test_labels.append(labels.cpu().numpy())



100%|██████████| 1092/1092 [09:42<00:00,  1.88it/s]


In [24]:
test_preds = np.concatenate(test_preds)
test_labels = np.concatenate(test_labels)

# Calculate metrics
test_preds_class = np.argmax(test_preds, axis=1)
accuracy = accuracy_score(test_labels, test_preds_class)
recall = recall_score(test_labels, test_preds_class)
precision = precision_score(test_labels, test_preds_class)
f1 = f1_score(test_labels, test_preds_class)
roc_auc = roc_auc_score(test_labels, test_preds[:, 1])

print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Roc Auc: {roc_auc}')


Accuracy: 0.901954795357361, Recall: 0.21560975609756097, Precision: 0.5815789473684211, F1: 0.3145907473309608, Roc Auc: 0.8320904853927475
